## 準備

In [1]:
import pandas as pd
from pyspark.sql import SparkSession
import pyspark
from pyspark.sql.functions import col, countDistinct, when, broadcast, count, row_number, substring,udf,dense_rank,format_string,first,lit, to_date, transform,collect_set
from pyspark.sql.window import Window
import unicodedata
from pyspark.sql.types import StringType
import numpy as np
import pyspark.sql.functions as F


# Sparkセッションがまだ存在しない場合に作成
import sys
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
    .appName("LoadParquetFile")
    .config("spark.executor.memory", "32g")
    .config("spark.driver.memory", "16g")
    .config("spark.sql.shuffle.partitions", "500")
    .config("spark.sql.files.maxPartitionBytes", "32m")
    # ここを追加（Driver/Executor ともに同じ Python を使わせる）
    .config("spark.pyspark.driver.python", sys.executable)
    .config("spark.pyspark.python", sys.executable)
    .getOrCreate()
)


file_path = "/Users/takami.soshi/Documents/analytics/chart_disease_merge/final_chart_patient_disease_with_icd.parquet"
df = spark.read.parquet(file_path)

soap_path = "/Users/takami.soshi/Documents/analytics/SOAP/preprocessed_soap"
soap_df = spark.read.parquet(soap_path)

df_lab = spark.read.parquet("/Users/takami.soshi/Documents/analytics/data2/data_for_model_with_dz_ord_by_chart_and_ordDisease.parquet")

ch_pat_path = "/Users/takami.soshi/Documents/analytics/row/chart_patient.csv"
ch_pat_df = spark.read.option("header",True).csv(ch_pat_path)


order_path = "/Users/takami.soshi/Documents/analytics/order_data_processed/grouped_recipe_shinsatsu_filtered.parquet"
order_df = spark.read.parquet(order_path)
order_df = order_df.filter(F.col("shinsatsu") == 1) 
order_df = order_df.drop("create_datetime")

@F.udf("string")
def norm(s):
    if s is None:
        return None
    s = unicodedata.normalize("NFKC", s).strip().lower()
    # 追加の正規化ルールがあればここで
    return s or None

25/11/07 10:55:20 WARN Utils: Your hostname, Donuts-NM3044-2508.local resolves to a loopback address: 127.0.0.1; using 10.10.35.188 instead (on interface en0)
25/11/07 10:55:20 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/07 10:55:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
import json

with open("/Users/takami.soshi/Documents/analytics/KG/医療シソーラス/synonym_to_icd.json", "r", encoding="utf-8") as f:
    synonym_to_icd = json.load(f)

from pyspark.sql import functions as F
df = spark.read.parquet("/Users/takami.soshi/Documents/analytics/project/shosin_disease_pred/shoshin_ch_ord_dis_soap_tok.parquet")
from pyspark.sql import functions as F, Window

node_df = spark.read.csv("/Users/takami.soshi/Documents/analytics/KG/cytoscape/aliases.csv", header=True, inferSchema=True)
df_tok = (
    spark.read
         .parquet("/Users/takami.soshi/Documents/analytics/project/shosin_disease_pred/shoshin_ch_ord_dis_soap_tok.parquet")
         .select("chart_id", "soap_tokens")
)


In [3]:
from pyspark.sql import functions as F

# 正規化 → explode
tok_norm = (
    df_tok
    .select(
        "chart_id",
        F.when(F.col("soap_tokens").isNull(), F.array().cast("array<string>"))
         .otherwise(F.col("soap_tokens")).alias("soap_tokens")
    )
    .select(
        "chart_id",
        F.expr("transform(soap_tokens, x -> trim(lower(x)))").alias("soap_tokens")
    )
)

tok_expl = (
    tok_norm
    .select("chart_id", F.explode("soap_tokens").alias("token"))
    .filter(F.col("token").isNotNull())
    .select("chart_id", F.col("token").cast("string").alias("token"))
)

# 一度だけ保存（パスは任意）
out_path = "/Users/takami.soshi/Documents/analytics/project/shosin_disease_pred/tok_expl.parquet"
# tok_expl.write.mode("overwrite").parquet(out_path)

# 以後は読み直して使う
tok_expl_saved = spark.read.parquet(out_path)
# tok_expl_saved.show(10, truncate=False)

# 2) token ごとに代表 chart_id を1つだけ残す（ここでは最小値）
tok_dedup = (
    tok_expl_saved
      .groupBy("token")
      .agg(F.min("chart_id").alias("chart_id"))
)

# tok_dedup.show(20, truncate=False)

# # 保存するなら
out_path = "/Users/takami.soshi/Documents/analytics/project/shosin_disease_pred/tok_unique_token.parquet"
# tok_dedup.write.mode("overwrite").parquet(out_path)

tok_dedup = spark.read.parquet("/Users/takami.soshi/Documents/analytics/project/shosin_disease_pred/tok_unique_token.parquet")
tok_dedup.show(10, truncate=False)


+--------------+--------+
|token         |chart_id|
+--------------+--------+
|r1            |22318792|
|倦怠感        |22417498|
|2回目         |22418736|
|なっ          |21663439|
|ホルネル症候群|22428691|
|日々          |22423552|
|serm          |22434945|
|抗凝固薬      |21963424|
|皮膚乾燥      |22426470|
|フラフラ      |22428657|
+--------------+--------+
only showing top 10 rows



## analysis

In [4]:
matched_token = spark.read.parquet("/Users/takami.soshi/Documents/analytics/project/shosin_disease_pred/soap_tokens_with_matched_aliases.parquet")
label = spark.read.parquet("/Users/takami.soshi/Documents/analytics/project/shosin_disease_pred/shoshin_ch_ord_dis_soap_clean_matched_icd.parquet")


In [7]:
matched_token.printSchema()

root
 |-- chart_id: integer (nullable = true)
 |-- soap_tokens: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- matched_aliases: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- matched_count: integer (nullable = true)
 |-- has_match: boolean (nullable = true)



In [5]:
matched_token.show()

+--------+----------------------------+-------------------------------+-------------+---------+
|chart_id|                 soap_tokens|                matched_aliases|matched_count|has_match|
+--------+----------------------------+-------------------------------+-------------+---------+
|22520504|     [(, s, ), 咳嗽, 、, ...|  [咽頭痛, 軽度発赤, 咳嗽, p...|            5|     true|
|22795670|[主訴, :, 咽頭違和感, ・,...| [黒色便, 胃痛, 排便問題, 食...|           15|     true|
|22962710|     [8, /, 1, 健, 診, で...| [血便, 子宮病変, 出血量, ふ...|            7|     true|
|22993800|[診察, 医, 署名, ：, 加納...|    [P, 症状, 湿疹, 癜風, 妊...|            6|     true|
|23075094| [S, ）, かわり, ない, で...|                             []|            0|    false|
|23131962|[薬, 希望, 。, 病状, 安定...|               [食欲不振, 不眠]|            2|     true|
|23213491| [【, 初診, 時, 記録, 】,...| [高血圧, 亜鉛, 妊娠中, 脂質...|           17|     true|
|23229925|        [&, lt, ;, S, &, ...|    [鼻水, P, 症状, 鶏眼, く...|           17|     true|
|23488054|       [S, ）, 8, /, 31, ...|    [風邪, 

In [9]:
from pyspark.sql import functions as F

# 文字前後の空白を削り、lower化し、空文字は落とす
normed = matched_token.withColumn(
    "matched_aliases_norm",
    F.expr("filter(transform(matched_aliases, x -> lower(trim(x))), x -> x <> '')")
)

# 1) 行ごとの要素数
with_len = normed.select(
    "*",
    F.coalesce(F.size("matched_aliases_norm"), F.lit(0)).alias("matched_aliases_len")
)
with_len.show(20, truncate=False)

# 2) 要素の頻度表
alias_freq = (
    normed
      .select(F.explode_outer("matched_aliases_norm").alias("alias"))
      .groupBy("alias")
      .count()
      .orderBy(F.desc("count"))
)
alias_freq.show(100, truncate=False)


+--------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

+-------------------------------------+-------+
|alias                                |count  |
+-------------------------------------+-------+
|p                                    |1298177|
|症状                                 |1197907|
|アレルギー                           |970365 |
|よう                                 |699027 |
|体重                                 |614364 |
|発熱                                 |610944 |
|咳                                   |541099 |
|妊娠                                 |512860 |
|痛み                                 |392847 |
|g                                    |377589 |
|疾患                                 |372652 |
|鼻水                                 |338321 |
|頭痛                                 |298680 |
|咽頭痛                               |298152 |
|増悪                                 |270645 |
|covid19                              |266633 |
|重症化                               |263601 |
|コロナ                               |241878 |
|病気                        

In [10]:
alias_freq

DataFrame[alias: string, count: bigint]

In [6]:
label.select("chart_id", "main_name_list", "suspect_name_list", "acute_name_list", "treat_name_list","soap_text").show()

+--------+------------------+--------------------------------+---------------+-----------------------------------+-------------------------------------+
|chart_id|    main_name_list|               suspect_name_list|acute_name_list|                    treat_name_list|                            soap_text|
+--------+------------------+--------------------------------+---------------+-----------------------------------+-------------------------------------+
|22418751|            [胃炎]|[肝機能障害, 腎機能低下, 脂質...|             []|                 [胃腸運動機能障害]|・ここ数日で発熱はなし　胃腸の調子...|
|22419784|    [急性気管支炎]|                    [細菌性肺炎]|             []|                                 []|【主訴】倦怠感、背部痛、下痢【現病...|
|22421836|                []|[インフルエンザ, インフルエンザ]|             []|          [ＣＯＶＩＤ−１９, 咽頭炎]|      \n### 症状を教えてください（...|
|22422296|                []|                              []|     [急性胃炎]|                     [感染性胃腸炎]|    7/27に外出して熱中症気味になっ...|
|22423019|      [急性咽頭炎]|                              